Column 1: “seqid” Accession.version of the annotated genomic sequence. NCBI files universally use accession.version because it provides an unambiguous identifier for the annotated sequence, and does not require additional knowledge of the species, assembly and version, and data source. We strongly recommend using accession.version instead of ambiguous seqids such as ‘chr1’ to avoid errors due to mis-associating features with the wrong genomic location.

Column 2: “source” For annotations produced by one of NCBI’s pipelines, the method used to generate the annotation is provided in column 2. The method is found in the ModelEvidence object in ASN.1 format, and appears in the flatfile format as a structured note. For example: “Derived by automated computational analysis using gene prediction method: BestRefSeq”

The reported methods for RefSeq eukaryotic annotations include:

BestRefSeq: feature projected from the alignment of a “known” RefSeq transcript to the genome

Curated Genomic: feature projected from the alignment of a curated RefSeq genomic sequence to the genome

Gnomon: feature predicted by Gnomon, using transcript and protein evidence and/or ab initio

BestRefSeq,Gnomon: gene with children features predicted by BestRefSeq and Gnomon

Curated Genomic,Gnomon: gene with children features predicted Curated Genomic or Gnomon

tRNAscan-SE: feature predicted by tRNAscan-SE

The reported methods for RefSeq prokaryotic annotations include:

GeneMarkS+: feature predicted by GeneMarkS+

Protein Homology: feature predicted by protein alignment

cmsearch: feature predicted by cmsearch

tRNAscan-SE: feature predicted by tRNAscan-SE

If the annotation method is not available, the source column is based on the source database for the record (RefSeq, GenBank, EMBL, DDBJ).

Column 3: “type” The SOFA feature type most equivalent to the feature found in the source annotation. The original GenBank feature type is also provided by the “gbkey” attribute in column 9.

Columns 4 & 5: “start” and “end” Start and end coordinates of the feature in 1-based coordinates. Note two exon or CDS rows of the same feature may overlap or be separated by an artificial “micro-intron” in order to represent cases of ribosomal slippage or putative assembly errors. See Additional Details below for more information.

Column 6: “score” Currently only provided for alignments, if they contain a score named “score”. The definition of this score may vary depending on the type of alignment.

Column 7: “strand” The strand of the feature

Column 8: “phase” The phase of the CDS feature, which is related to /codon_start in the flatfile specification. The phase is computed based on the known phase at the start of the CDS and computed for subsequent CDS rows. It may not be accurate if the CDS contains internal frameshifts, which can occur in pseudogenes and in genomes with indels, assembly gaps, and other errors. See Additional Details below for more information.

Column 9: “attributes” A semicolon delimited list of official and additional attributes describing the feature.

Credit: https://www.ncbi.nlm.nih.gov/datasets/docs/v1/reference-docs/file-formats/about-ncbi-gff3/

In [47]:
import pandas as pd
import tqdm 
from sqlalchemy import URL,engine
from config_parse import configargs

In [32]:
def parse_gff3(fpath):
    """Parses GFF3 files and extracts both gene and transcript info into seperate dataframes

    Args:
        fpath (str): File path to GFF3 file

    Returns:
        gene_df (pandas.core.frame.DataFrame): Dataframe containg gene information
        transcript_df (pandas.core.frame.DataFrame): Dataframe containg transcript information containg seperate exons
    """
    # Initialize empty lists to store gene and transcript data
    gene_data = []
    transcript_data = []
    
    # Open the GFF3 file
    with open(fpath) as gff3:
        for line in tqdm.tqdm(gff3):
            # Skip header lines
            if line.startswith("#"):
                continue

            # Split the GFF3 columns
            cols = line.strip().split('\t')
            seqid, source, feature_type, start, end, score, strand, phase, attributes = cols
            
            # Parse attributes into a dictionary
            attr_dict = {}
            for attr in attributes.split(';'):
                key, value = attr.split('=')
                attr_dict[key] = value
            
            # If the feature is a gene, populate the gene table
            if feature_type == 'gene':
                gene_id = attr_dict.get('ID', '')
                gene_name = attr_dict.get('gene_name', '')
                gene_type = attr_dict.get('gene_type', '')
                
                gene_data.append({
                    'gene_id': gene_id,
                    'gene_name': gene_name,
                    'chromosome': seqid,
                    'start_position': int(start),
                    'end_position': int(end),
                    'strand': strand,
                    'gene_type': gene_type
                })
            
            # If the feature is a transcript or exon, populate the transcript table
            elif feature_type in ['transcript', 'exon']:
                transcript_id = attr_dict.get('ID', '')
                
                # Remove 'exon:' prefix if present
                if 'exon:' in transcript_id:
                    transcript_id = transcript_id.replace('exon:', '')
                
                gene_id = attr_dict.get('Parent', '')  # Links to the gene
                
                # For exons, you'll also want to store exon-specific info
                exon_count = None
                if feature_type == 'exon':
                    exon_count = int(attr_dict.get('exon_number', 0))
                
                transcript_data.append({
                    'transcript_id': transcript_id,
                    'gene_id': gene_id,
                    'start_position': int(start),
                    'end_position': int(end),
                    'exon_count': exon_count,
                    'strand': strand,
                    'score': score
                })
    
    # Convert lists to DataFrames for easy manipulation and database insertion
    gene_df = pd.DataFrame(gene_data)
    transcript_df = pd.DataFrame(transcript_data)
    
    return gene_df, transcript_df

In [33]:
fpath = r"D:\GitHub\variant-db-postgreSQL\data\GRCh38.p14.basic.annotation.gff3"
gene_df, transcript_df = parse_gff3(fpath)

2223398it [00:14, 152134.99it/s]


In [48]:
try:
    db_type,db_host,db_name,db_user,db_password,db_port = configargs("config.json")
except FileNotFoundError as e:
    print("Config file not found, please place in main directory")

KeyError: 'postgreSQL'

In [43]:
url_object = URL.create(
    "postgresql",
    username=db_user,
    password=db_password,  
    host=db_host,
    database=db_name,
    port=db_port
)

In [ ]:
engine = engine.create_engine(url_object)